## Zajęcia 9: Linear regression

Wszystkie zadania ćwiczeniowe należy rozwiązywać w języku Python w kopii Jupyter Notebook'a dla danych zajęć w wyznaczonych miejscach (komórki z komentarzem `# Solution`).

Nie należy usuwać komórek z treścią zadań.

Należy wyświetlać outputy przy pomocy `print`

### Zadanie 1

Na podstawie danych z wyzwania https://amueval.pl/challenge/Tripadvisor%20Review%20Prediction wytrenuj model regresji liniowej (można skorzystać z gotowej implementacji z biblioteki scikit-learn. Dokonaj inferencji modelu na zbiorze testowym i wrzuć rozwiązanie na platformę AmuEval.

Wybór cech modelu (one-hot encoding/frequency encoding/TF-IDF encoding lub może coś innego) pozostaje otwarty.
Losowy submit posiada RMSE (root-mean-square-error) na poziomie około 2.0, także rozwiązania o podobnym wyniku nie będą akceptowane.

# INDEKS: 481825

## Solution

In [112]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression

import pandas as pd
import numpy as np

### 1. Reading data and first preprocessing

In [113]:
def read_data(filename):
    data=pd.read_csv(filename, sep='\t',header=None)
    return data
    

In [114]:
reviews=read_data("tripadvisor-review-prediction/train/in.tsv")
reviews.columns=['review']
labels=read_data("tripadvisor-review-prediction/train/expected.tsv")
labels.columns=['label']
Y=labels

### 2. Train reviews encoding.

In [115]:
def tfidf_fit(data):
    vectorizer=TfidfVectorizer(
            lowercase=True,
            stop_words='english',
            max_features=8000,
            min_df=0.002,
            max_df=0.8)
    vectorizer=vectorizer.fit(data)
    return vectorizer

def tfidf_encode(data,vectorizer):
    X=vectorizer.transform(data)
    encoded_messages=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
    return encoded_messages

vectorizer=tfidf_fit(reviews['review'])

X=tfidf_encode(reviews['review'],vectorizer)

3. Model training

In [116]:
linear_model=LinearRegression()
linear_model.fit(X,Y)
print("training done")


training done


4. Testing data prediction

In [117]:
test=read_data("tripadvisor-review-prediction/test/in.tsv")
test.columns=['review']
encoded_test_X=tfidf_encode(test['review'],vectorizer)

test_predicted_output=linear_model.predict(encoded_test_X)

test_predicted_output_rounded=np.round(test_predicted_output)
test_predicted_output_rounded=np.clip(test_predicted_output_rounded,1,5)

test_predicted_output_rounded=pd.DataFrame(test_predicted_output_rounded,columns=['label'])

test_predicted_output_rounded.to_csv("tripadvisor-review-prediction/test/out.tsv",sep='\t',header=False,index=False)